#### Extending the P0 Library

The earlier implementation of the random number generator will always produce the same sequence as the seed value is fixed. Modify the implementation to use the current time in some form as the seed value:

```Pascal
var r: integer

procedure randint(bound: integer) → (rand: integer)
  const a = 16807
  const c = 11
  const m = 65535
    r := (a × r + c) mod m
    rand := r div bound

procedure gcd(x: integer; y: integer) → (d: integer)
  while x ≠ y do
    if x > y then x := x - y
    else y := y - x
  d := x

program randgcd
  var x, y, d: integer
    r ← seed()
    x ← randint(100); write(x)
    y ← randint(100); write(y)
    d ← gcd(x, y); write(d)
```

For this, you must extend the standard library with a function that returns the current time. Note that that value cannot be too large as otherwise the multiplication with `a` may overflow. For example, the millisecond portion of the current time works.

In [4]:
def runpywasm(wasmfile):
    import pywasm
    from datetime import datetime
    def write(s, i): print(i, end=' ')
    def writeln(s): print()
    def read(s): return int(input())
    def seed(s): return datetime.now().microsecond // 1000
    vm = pywasm.load(wasmfile, {'P0lib': {'write': write, 'writeln': writeln, 'read': read, 'seed': seed}})

In [5]:
%%writefile randgcd2.wat
(module
 (import "P0lib" "write" (func $write (param i32)))
 (import "P0lib" "writeln" (func $writeln))
 (import "P0lib" "read" (func $read (result i32)))
 (import "P0lib" "seed" (func $seed (result i32)))
 (global $r (mut i32) (i32.const 0))
 (func $randint (param $bound i32) (result i32)
    (local $rand i32)
    (local $a i32)
    (local $c i32)
    (local $m i32)
    i32.const 16807
    local.set $a
    i32.const 11
    local.set $c
    i32.const 65535
    local.set $m
  
    local.get $c
    local.get $a
    global.get $r
    i32.mul
    i32.add
    local.get $m
    i32.rem_s
    global.set $r
  
    global.get $r
    local.get $bound
    i32.div_s
    local.set $rand
    local.get $rand
 )
 (func $gcd (param $x i32) (param $y i32) (result i32)
    (local $d i32)
    loop $label0
      local.get $x
      local.get $y
      i32.ne
      if
        local.get $x
        local.get $y
        i32.gt_s
        if
          local.get $x
          local.get $y
          i32.sub
          local.set $x
        else
          local.get $y
          local.get $x
          i32.sub
          local.set $y
        end
        br $label0
      end
    end
    local.get $x
    local.set $d
    local.get $d
 )
 (func $randgcd
    (local $x i32)
    (local $y i32)
    (local $d i32)
  
    call $seed
    global.set $r

    i32.const 100
    call $randint
    local.set $x
    local.get $x
    call $write

    i32.const 100
    call $randint
    local.set $y
    local.get $y
    call $write

    local.get $x
    local.get $y
    call $gcd
    local.set $d
    local.get $d
    call $write
 )
 (memory 1)
 (start $randgcd)
)

Overwriting randgcd2.wat


In [6]:
!wat2wasm randgcd2.wat

In [8]:
runpywasm("randgcd2.wasm")

383 135 1 